Generally following:<br>
https://medium.com/data-from-the-trenches/text-classification-the-first-step-toward-nlp-mastery-f5f95d525d73<br>
https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

In [61]:
import pprint
import re
import glob
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [2]:
text_files = glob.glob('C:\\UT\\DataMining\\books\\set0\\*.txt')

Unreadable character in western15.txt gave me trouble, so I renamed it western15.txt5<br>
so it wouldn't get included.

In [3]:
cutoff = 9
mstr = '\\'
train_texts = []
test_texts = []
train_cats = []
test_cats = []
for file in text_files:
    fileNum = int(file[-6:-4])
    v = file.rfind(mstr) + len(mstr)
    fileCat = file[v:-6]
    if (fileNum <= cutoff):
        with open(file, 'r', encoding='utf-8') as f:
            text = f.read()
            train_texts.append(text)
            train_cats.append(fileCat)
    else:
        with open(file, 'r', encoding='utf-8') as f:
            text = f.read()
            test_texts.append(text)
            test_cats.append(fileCat)

In [4]:
print(len(train_texts), len(test_texts))
print(len(train_cats), len(test_cats))

68 55
68 55


In [5]:
df_train = pd.DataFrame(np.array(train_texts))
df_train['Category'] = train_cats
df_train.columns = ["text", "category"]

In [6]:
df_train.head()

,text,category
0,﻿The Project Gutenberg eBook of The Count of M...,adventure
1,﻿\nThe Project Gutenberg EBook of The Call of ...,adventure
2,﻿The Project Gutenberg EBook of Around the Wor...,adventure
3,"﻿The Project Gutenberg eBook, White Fang, by J...",adventure
4,﻿ THE THREE MUSKETEER...,adventure


In [7]:
df_test = pd.DataFrame(np.array(test_texts))
df_test['Category'] = test_cats
df_test.columns = ["text", "category"]

In [8]:
df_test.head()

,text,category
0,﻿The Project Gutenberg EBook of King Solomon's...,adventure
1,"﻿The Project Gutenberg eBook of Moby-Dick, by ...",adventure
2,﻿The Project Gutenberg EBook of The Prisoner o...,adventure
3,"﻿The Project Gutenberg eBook of She, by H. Rid...",adventure
4,"﻿Project Gutenberg’s The Man in the Iron Mask,...",adventure


In [9]:
def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """
    
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)
    
    # remove numbers including unsigned / signed integers and floats
    text = re.sub(r"[0-9]+", "", text)
    text = re.sub(r"[-0-9]+", "", text)
    text = re.sub(r"[+.0-9]+", "", text)
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)
    
    # remove non-ascii characters
    text = re.sub(r'[^\x00-\x7F]+','', text)

    return text

In [31]:
# Transform each text into a vector of word counts
# CountVectorizer calls preprocessor, filters stopwords, and tokenizes to sparse matrix
# "Count" is a crude metric since longer books will naturally have more words
vectorizer = CountVectorizer(stop_words="english",
                             preprocessor=clean_text,
                             strip_accents="ascii")

train_features = vectorizer.fit_transform(df_train["text"])    
test_features = vectorizer.transform(df_test["text"])

# Training
model = LinearSVC()
model.fit(train_features, df_train["category"])
y_pred_0 = model.predict(test_features)

# Evaluation
acc = accuracy_score(df_test["category"], y_pred_0)

print("Accuracy: {:.2f}".format(acc*100))

Accuracy: 56.36


In [37]:
#dictionary of feature indices, it's length is the number of features (words) in the data set
print(len(vectorizer.get_feature_names()))

95014


In [22]:
# Transform each text into a vector of word counts
# TF (Term Frequency) times IDF (Inverse Document Frequency) is better than a count vectorizer
# TF measures words by frequency to normalize for book length
# IDF reduces the weight of words that appear in many documents
vectorizer = TfidfVectorizer(stop_words="english",
                             preprocessor=clean_text,
                             ngram_range=(1, 2))

train_features = vectorizer.fit_transform(df_train["text"])
test_features = vectorizer.transform(df_test["text"])

# Training
model = LinearSVC()
model.fit(train_features, df_train["category"])
y_pred_1 = model.predict(test_features)

# Evaluation
acc = accuracy_score(df_test["category"], y_pred_1)

print("Accuracy with ngram: {:.2f}".format(acc*100))

Accuracy with ngram: 58.18


In [77]:
print('{0:<12} {1:<12} {2:<12} {3:<12}'.format("True", "Predicted1", "Predicted2", "Predicted4"))
print('{0:<12} {1:<12} {2:<12} {3:<12}'.format("----", "----------","----------", "----------"))
for i in range(len(y_pred_2)):
    print('{0:<12} {1:<12} {2:<12} {3:<12}'.format(df_test["category"][i], y_pred_1[i], y_pred_2[i], y_pred_4[i]))
    #print(df_test["category"][i], y_pred[i])

True         Predicted1   Predicted2   Predicted4  
----         ----------   ----------   ----------  
adventure    adventure    adventure    adventure   
adventure    adventure    scifi        adventure   
adventure    humor        adventure    adventure   
adventure    gothic       adventure    gothic      
adventure    adventure    adventure    adventure   
adventure    adventure    adventure    adventure   
adventure    adventure    adventure    adventure   
adventure    humor        adventure    adventure   
adventure    gothic       adventure    adventure   
adventure    mystery      adventure    adventure   
gothic       mystery      gothic       gothic      
gothic       horror       horror       horror      
gothic       horror       horror       horror      
gothic       mystery      mystery      horror      
gothic       adventure    adventure    adventure   
gothic       gothic       gothic       gothic      
horror       scifi        western      scifi       
horror      

In [39]:
confusion_matrix(df_test["category"], y_pred_0)

array([[5, 0, 3, 1, 1, 0, 0],
       [0, 2, 3, 0, 1, 0, 0],
       [0, 0, 4, 1, 0, 5, 0],
       [1, 2, 1, 5, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 0, 0, 8, 0],
       [1, 1, 0, 0, 0, 0, 7]], dtype=int64)

In [40]:
confusion_matrix(df_test["category"], y_pred_1)

array([[5, 2, 0, 2, 1, 0, 0],
       [1, 1, 2, 0, 2, 0, 0],
       [0, 0, 3, 1, 0, 6, 0],
       [1, 1, 0, 7, 0, 1, 0],
       [0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 9, 0],
       [1, 0, 0, 1, 0, 0, 7]], dtype=int64)

In [43]:
clf = MultinomialNB().fit(train_features, df_train["category"])
y_pred_2 = clf.predict(test_features)

# Evaluation
acc = accuracy_score(df_test["category"], y_pred_2)

print("Accuracy: {:.2f}".format(acc*100))

Accuracy with ngram: 63.64


In [52]:
text_clf = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                         alpha=1e-3, random_state=42,
                         max_iter=5, tol=None)),
])

In [58]:
text_clf.fit(df_train["text"], df_train["category"])
y_pred_3 = text_clf.predict(df_test["text"])

# Evaluation
acc = accuracy_score(df_test["category"], y_pred_3)

print("Accuracy: {:.2f}".format(acc*100))

Accuracy: 27.27


In [60]:
print(classification_report(df_test["category"], y_pred_3))c

              precision    recall  f1-score   support

   adventure       0.24      1.00      0.39        10
      gothic       0.00      0.00      0.00         6
      horror       0.00      0.00      0.00        10
       humor       0.50      0.10      0.17        10
     mystery       0.00      0.00      0.00         0
       scifi       1.00      0.10      0.18        10
     western       0.50      0.33      0.40         9

    accuracy                           0.27        55
   macro avg       0.32      0.22      0.16        55
weighted avg       0.40      0.27      0.20        55



C:\Users\clayj\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\clayj\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [62]:
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'tfidf__use_idf': (True, False),
    'clf__alpha': (1e-2, 1e-3)
}

In [63]:
gs_clf = GridSearchCV(text_clf, parameters, cv=5, n_jobs=-1)

In [64]:
gs_clf = gs_clf.fit(df_train["text"], df_train["category"])

In [65]:
gs_clf.best_params_

{'clf__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}

***

In [66]:
text_clf_nb = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])

In [67]:
parameters = {
    'vect__ngram_range': [(1, 1), (1, 2)],
    'clf__fit_prior': (True, False),
    'clf__alpha': (0, 1e-2, 1e-3, 0.5, 1.0)
}

In [68]:
gs_clf_nb = GridSearchCV(text_clf_nb, parameters, cv=5, n_jobs=-1)

In [69]:
gs_clf_nb = gs_clf_nb.fit(df_train["text"], df_train["category"])

In [70]:
gs_clf_nb.best_params_

{'clf__alpha': 0.001, 'clf__fit_prior': True, 'vect__ngram_range': (1, 1)}

Best parameters:<br>
{'clf__alpha': 0.001, 'clf__fit_prior': True, 'vect__ngram_range': (1, 1)}

In [71]:
nb_best = gs_clf_nb.best_estimator_

In [72]:
y_pred_4 = nb_best.predict(df_test["text"])

# Evaluation
acc = accuracy_score(df_test["category"], y_pred_4)

print("Accuracy: {:.2f}".format(acc*100))

Accuracy: 69.09


In [78]:
print(classification_report(df_test["category"], y_pred_4))

              precision    recall  f1-score   support

   adventure       0.90      0.90      0.90        10
      gothic       0.22      0.33      0.27         6
      horror       0.33      0.20      0.25        10
       humor       1.00      0.70      0.82        10
       scifi       0.64      0.90      0.75        10
     western       1.00      1.00      1.00         9

    accuracy                           0.69        55
   macro avg       0.68      0.67      0.67        55
weighted avg       0.71      0.69      0.69        55



In [87]:
unique_label = list(df_test["category"].unique())
cmtx = pd.DataFrame(confusion_matrix(df_test["category"], y_pred_4, labels= unique_label),
                    index=['true:{:}'.format(x) for x in unique_label], 
                    columns=['pred:{:}'.format(x) for x in unique_label])
print(cmtx)

                pred:adventure  pred:gothic  pred:horror  pred:humor  \
true:adventure               9            1            0           0   
true:gothic                  1            2            3           0   
true:horror                  0            3            2           0   
true:humor                   0            2            1           7   
true:scifi                   0            1            0           0   
true:western                 0            0            0           0   

                pred:scifi  pred:western  
true:adventure           0             0  
true:gothic              0             0  
true:horror              5             0  
true:humor               0             0  
true:scifi               9             0  
true:western             0             9  
